In [ ]:
pip install deap

In [ ]:
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt
import random
import numpy as np
import math

In [ ]:
# TODO: Task 2 - Define the number of locations and vehicles

num_locations = 10# Define the number of locations (eg 10)

locations = [(random.uniform(0,100),random.uniform(0,100)) for _ in range(num_locations)]# Create a list of tuples representing location coordinates -
            # try to use a random number generator
            # (x, y) could be between 0 and 100

depot = (50,50)

num_vehicles = 4# Define the number of vehicles - (eg 3)

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(num_locations), num_locations)
toolbox.register("individual", tools.initIterate,creator.Individual,toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [ ]:
# HINT CODE

def evalVRP(individual):
    total_distance = 0
    distances = []  # Track distance traveled by each vehicle for balance calculation
    # Split the list of locations among vehicles, ensuring each starts and ends at the depot
    for i in range(num_vehicles):

        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]
        vehicle_distance=0
        for i in range(len(vehicle_route)):
          a=vehicle_route[i]
          b=vehicle_distance[i+1]
          vehicle_distance+=math.sqrt((x[0]-y[0])**2+(x[1]-y[1])**2)



        total_distance += vehicle_distance
        distances.append(vehicle_distance)

    balance_penalty = np.std(distances)  # Use standard deviation of distances as a penalty for imbalance among vehicles
    return total_distance, balance_penalty

toolbox.register("evaluate", evalVRP)

In [ ]:
#SOLUTION CODE


#Fitness Function
def evalVRP(individual):
    total_distance = 0
    distances = []  # Track distance traveled by each vehicle for balance calculation
    # Split the list of locations among vehicles, ensuring each starts and ends at the depot
    for i in range(num_vehicles):

        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]

        # Calculate total distance traveled by this vehicle
        vehicle_distance = sum(np.linalg.norm(np.array(vehicle_route[k+1]) - np.array(vehicle_route[k])) for k in range(len(vehicle_route)-1))

        total_distance += vehicle_distance
        distances.append(vehicle_distance)

    balance_penalty = np.std(distances)  # Use standard deviation of distances as a penalty for imbalance among vehicles
    return total_distance, balance_penalty

In [ ]:
toolbox.register("mate", tools.cxPartialyMatched)  # Register the crossover function suitable for permutation-based representation
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)  # Register the mutation function to shuffle indices with a 5% chance per index
toolbox.register("select", tools.selTournament, tournsize=3)  # Register the selection function using tournament selection

In [ ]:
# Plotting Function
def plot_routes(individual, title="Routes"):
   plt.figure(figsize=(8, 8))
   for i in range(len(individual) - 1):
       x = locations[individual[i]]
       y = locations[individual[i + 1]]
       plt.plot([x[0], y[0]], [x[1], y[1]], 'b-')
   plt.scatter([loc[0] for loc in locations], [loc[1] for loc in locations], color='r')
   plt.scatter([depot[0]], [depot[1]], color='g')
   plt.title(title)
   plt.show()

In [34]:
def main():
    random.seed(42)
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)

    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 300, stats=stats, halloffame=hof)
    plot_routes(hof[0], "Optimal Route")
    return pop, stats, hof
    if _name_ == "_main_":
      main()